# 1. 개요 (Overview)

* Last updated: 2019-10-16 (Wed)
* First written: 2019-10-16 (Wed)
* Tae-Hyung Kim, Ph.D. 

## 1.1. 목표 (Objectives)
* 인공지능과 자연어처리 (Natural Language Processing)를 공부하기 위해서
  * 텍스트 처리 (Text Processing)의 한 분야인 감성 분석 (Sentiment Analysis) 문제를 다뤄봅니다.
  * 감성 분석을 수행하는 코드를 이해하고 결과를 재생산 (Reproduce)하기 위해 필요한 지식을 습득합니다.
  * 필요한 지식을 이해하기 위한 배경지식을 습득합니다.
* 이 과정에서
  * 인공지능의 한 분야인 
    * 기계학습 (혹은 머신러닝 Machine Learning)과
    * 심층학습 (혹은 딥러닝 Deep Learning)
  * 의 개념과 일부 기술을 이해하고 필요한 기술을 습득합니다.


## 1.2. 결과 예시: 소스 코드 샘플 (Source Code Sample)
아래의 소스 코드는
* Keras로 IMDB Movie Review Dataset에 대한 감성 분석을 하는 간단한 예제 코드입니다.
* RNN의 한 종류인 LSTM으로 데이터셋을 훈련 (Training)하고 테스트 (Test) 합니다.
* 학습된 딥러닝 모델로 유사한 Movie Review Data에 대한 결과 예측 (Prediction)이 가능합니다.

```python
import os
import json

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf

#%%##################
# Directory & Files #
#####################
dir_data_in        = '../data_in/'
dir_data_out       = '../data_out/'
dir_checkpoint_rnn = '../checkpoint/rnn'

if not os.path.exists( dir_data_out ):
    os.makerdirs( dir_data_out )

# The input & label data were saved to a Numpy file.
file_train_input_data = 'train_input.npy'
file_train_label_data = 'train_label.npy'
file_test_input_data  = 'test_input.npy'
file_test_id_data     = 'test_id.npy'

file_data_configs     = 'data_configs.json'
file_result_output    = 'movie_review_result-rnn.csv'
#%%##############
# Load the data #
#################
# The preprocessed data was saved to numpy files (binary)
file = dir_data_out + file_train_input_data
with open( file, 'rb') as f:
    input_data = np.load( f )
    print(f'Loaded from {file}...')

file = dir_data_out + file_train_label_data
with open( file, 'rb') as f:
    label_data = np.load( f )
    print(f'Loaded from {file}...')

# The vocaburary and stuff...
#prepro_configs = None
file = dir_data_out + file_data_configs
with open( file, 'r') as f:
    prepro_configs = json.load( f )

#>>> prepro_configs
#'data_configs.json'

#%%###############
# Configurations #
##################
random_seed      = 13371447
test_split_ratio = 0.1  # TODO:Why is this changed to 0.1 from 0.2?
batch_size       = 16
num_epochs       = 3

#%%###################
# Training the Model #
######################
# Split training and testing data
train_input, test_input, train_label, test_label = train_test_split( input_data, label_data, test_size=test_split_ratio, random_state=random_seed )
# train_input, eval_input, train_label, eval_label = train_input, test_input, train_label, test_label
# train_data_features, y = input_data, label_data

def mapping_fn(x, y):
    inputs, labels = {'x': x}, y
    return inputs, labels

def train_input_fn():
    dataset  = tf.data.Dataset.from_tensor_slices( (train_input, train_label) )
    dataset  = tf.data.shuffle( buffer_size=50000 )
    dataset  = tf.data.batch( batch_size )
    dataset  = tf.data.repeat( count=num_epochs )
    dataset  = tf.data.map( mapping_fn )
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset  = tf.data.Dataset.from_tensor_slices( (test_input, train_label) )
    dataset  = tf.data.batch( batch_size )
    dataset  = tf.data.map( mapping_fn )
    #dataset  = tf.data.batch( batch_size*2 )  # TODO: why *2  and AFTER map?
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

#%%###################
# Training the Model #
######################
#print( type( prepro_configs['vocab_size'] ) )
print( int( prepro_configs['vocab_size'] ) )

vocab_size        = prepro_configs['vocab_size'] + 1  # +1 is for 'unk' or unknown
# TypeError: string indices must be integers

word_embeding_dim = 100
hidden_state_dim  = 150
dense_feature_dim = 150
drop_out_rate     = 0.2
learning_rate     = 0.001

# Double check the result
print( len(prepro_configs['vocab']), vocab_size )
# TODO: assert?

def model_fn( features, labels, mode ):
    # True if the condition is met
    training_mode   = ( mode == tf.estimator.ModeKeys.TRAIN )    
    testing_mode    = ( mode == tf.estimator.ModeKeys.EVAL )
    prediction_mode = ( mode == tf.estimator.ModeKeys.PREDICT )
    
    input_data      = features['x']
    embedding_layer = tf.keras.layers.Embedding( vocab_size, word_embeding_dim)( input_data )
    rnn_layers      = [tf.nn.rnn_cell.LSTMCell(size) for size in [hidden_state_dim, hidden_state_dim]]
    multi_rnn_cell  = tf.nn.rnn_cell.MultiRNNCell( rnn_layers )
    outputs, state  = tf.nn.dynamic_rnn( cell=multi_rnn_cell,
                                         inputs=embedding_layer,
                                         dtype=tf.float32 )
    
    outputs          = tf.keras.layers.Dropout( drop_out_rate )( outputs )
    hidden_layer     = tf.keras.layers.Dense( dense_feature_dim,
                                              activation=tf.nn.tanh )( outputs[:,-1,:] )
    hidden_layer     = tf.keras.layers.Dropout( drop_out_rate )( hidden_layer )
    
    logits           = tf.keras.layers.Dense(1)( hidden_layer )
    logits           = tf.squeeze( logits, axis=-1 )
    sigmoid_logits   = tf.nn.sigmoid( logits )
    
    # TODO: I have to double-check if logits are the right one to use!
    loss             = tf.losses.sigmoid_cross_entropy( labels, logits )
    
    if training_mode:
        global_step = tf.train.get_global_step()
        train_op    = tf.train.AdamOptimizer( learning_rate ).minimize( loss, global_step )
        
        return tf.estimator.EstimatorSpec( mode=mode, train_opt=train_op, loss=loss )
    
    if testing_mode:
        classes         = tf.round( sigmoid_logits)
        accuracy        = tf.metrics.accuracy( labels, classes )
        eval_metric_ops = {'acc': accuracy}  # TODO: or acc-> accuracy
        
        return tf.estimator.EstimatorSpec( mode=mode, loss=loss, eval_metric_ops = eval_metric_ops )
    
    if prediction_mode:
        predictions = {'sentiment': sigmoid_logits }
        
        return tf.estimator.EstimatorSpec( mode=mode, predictions=predictions )

#%%#################
# Train & Evaluate #
####################
os.environ['CUDA_VISIBLE_DEVICES']='4'

directory = dir_data_out + dir_checkpoint_rnn
est = tf.estimator.Estimator( model_fn=model_fn, model_dir=directory)
est.train( train_input_fn )
est.evaluate( eval_input_fn )

#%%##############
# Load the data #
#################
# The preprocessed data was saved to numpy files (binary)
file = dir_data_out + file_test_input_data
with open( file, 'rb') as f:
    test_input_data = np.load( f )
    print(f'Loaded from {file}...')

predict_input_fn = tf.estimator.inputs.numpy_input_fn( x={'x':test_input_data}, shuffle=False )

predictions = np.array( [p['sentiment'] for p in est.predict( input_fn=predict_input_fn)] )

#%%##################
# Saving the Result #
#####################

file = dir_data_out + file_test_id_data
with open( file, 'rb') as f:
    test_id = np.load( f )
    print(f'Loaded from {file}...')

# From dictionary to dataframe
result    = {'id': list(test_id), 'sentiment': list(predictions) }
result_df = pd.DataFrame( result )

file = dir_data_out + file_result_output
result_df.to_csv( file, index=False, quoting=3 )
print(f'Saved to {file}...')

```

## 1.3. 학습 이전에 필요한 절차들...
위의 코드는 훈련 데이터와 테스트 데이터가 주어진 상황에서 훈련과 테스트를 수행합니다. 그 이전에 수행되어야 할 데이터 준비 (Data Preparation) 과정들이 포함되지 않았습니다. 예를 들면, 아래의 절차는 생략했습니다.
* 탐색적 데이터 분석 (Exploratory Data Analysis or EDA)
* 데이터 전처리 (Data Preprocessing)

하지만 이런 과정은 훈련 및 테스트 데이터를 생성하기 위해 필요합니다. 이에 더해 딥러닝 모델의 성능은 
1. 적용되는 딥러닝 기술 (예: LSTM)과
2. 어떤 데이터셋으로 훈련하는지

에 의해 좌우됩니다. 그러므로 데이터 준비 과정을 위한 기본적인 기술을 이해하고 적용할 수 있는 능력은 의미가 있습니다.

왜냐하면 장난처럼 작은 문제 (Toy Problem)나 교실 안 문제 (Class room Problem)는 데이터가 주어지지만 실제로 프로젝트를 수행할 때는 (Real World Problem) 스스로 데이터를 준비해야 하기 때문에 데이터 준비 과정에 대한 이해는 딥러닝을 수행하는데 있어 필수적인 기술입니다.

## 1.4. 리뷰할 소스코드
이런 맥락에서 하나의 딥러닝 기법인 LSTM으로 감성분석 과제(Task)를 수행하는 모델을 만들기 위한 학습 과정에 있는 핵심 절차와 내용을 정리해봅니다. 참고로 감성분석은 텍스트 처리 (혹은 자연어 처리) 응용분야 중 하나입니다. 이 과제에 대한 이해를 확장하여 다른 응용분야의 과제들을 하나씩 도전하다보면, 텍스트 처리 (혹은 자연어 처리) 분야에 대한 전반적인 이해가 가능합니다. 이에 더해 머신러닝/딥러닝 모델의 코드는 재사용 (Code Reuse)이 가능합니다. 분야를 텍스트에서 이미지 (Image)나 음성 (Speech)에 확장할 때는 데이터 준비 및 인코딩 과정이 바뀝니다.

### 데이터 생성 (Data Generation)
* 1_1-exploratory_data_analysis-imdb_movie_review_dataset.py
* 1_2-data_preprocessing-imdb_movie_review_dataset.py

### 데이터 인코딩 (Data Encoding), i.e. 텍스트 인코딩 (Text Encoding)
* 2_1-vectorizing_text_with_tf-imdb_movie_review_dataset.py
  * from sklearn.feature_extraction.text import CountVectorizer
* 2_2-vectorizing_text_with_tf_idf-imdb_movie_review_dataset.py
  * from sklearn.feature_extraction.text import TfidfVectorizer
* 2_3-vectorizing_text_with_word_embedding-imdb_movie_review_dataset.py

TODO: 
1. Split 3-vectorizing_text_with_tf_idf-imdb_movie_review_dataset.py into two:
  * 4-vectorizing_text_with_tf_idf-imdb_movie_review_dataset.py and
  * 6-modeling_with_logistic_regression-imdb_movie_review_dataset.py
2. Extract a part of code to make "3-vectorizing_text_with_tf-imdb_movie_review_dataset.py"

### 모델 훈련 및 평가 (Training and Evaluation of Models)
#### 머신러닝 (Machine Learning)
* 3_1-modeling_with_logistic_regression-imdb_movie_review_dataset.py
* 3_2-modeling_with_random_forest-imdb_movie_review_dataset.py

#### 딥러닝 (Deep Learning)
* 3-modeling_with_rnn-imdb_movie_review_dataset.py
* modeling_with_cnn-imdb_movie_review_dataset.py

(EOF)